In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from IPython.display import display
%reload_ext autoreload
%autoreload 2

import shutil
usetex = True if shutil.which('latex') else False
print("found latex:", usetex)
plt.rcParams.update({
    "text.usetex": usetex,
    "font.family": "DejaVu Sans",
    "font.size": 12,
})
plt.rc('text.latex', preamble=r'\usepackage{bm}')
figsize = 7

#%matplotlib notebook
%matplotlib inline

from lifters.plotting_tools import *

# Noise study

In [ ]:
#df = pd.read_pickle("../_results/study_stereo1d_zero_noise.pkl")
import sys
sys.path.append("../_scripts/")
from noise_study import NOISE_DICT

import itertools

#d_list = [1, 2, 3]
d_list = [3]
noise_list = list(NOISE_DICT.keys())[:5]
level_list = [0, 3]
for d, level in itertools.product(d_list, level_list):
    if d == 1 and level == 3:
        continue
        
    fig, axs = plt.subplots(1, len(noise_list), sharey=True, sharex=True)
    fig.set_size_inches(len(noise_list)*3, 3)
    fig.suptitle(f"{d}d stereo, Lassere level {level}", y=1.0)
    for ax, noise in zip(axs, noise_list):
    
        name = f"study_stereo{d}d_{noise}noise_{level}level"
        try:
            df = pd.read_pickle(f"../_results/{name}.pkl")
        except:
            print(f"no results yet for {name}")
            continue
            
        if len(df) == 0:
            print(f"no data for {name}")
            continue

        plot_tightness(df, ax=ax)
        ax.legend([])
        ax.set_title(f"noise {NOISE_DICT[noise]:.0g}")
        ax.set_xlabel("constraints added")
    
    savefig(fig, f"../_plots/stereo_study_noise_{d}d_{level}level")

# Scaling study

In [ ]:
def poly_label(coeffs, xlabel="x"):
    label = ""
    for i, c in enumerate(coeffs):
        deg = len(coeffs) - i
        if deg > 1:
            if abs(c) > 1e-10:
                label += f"{c:.1f}${xlabel}^{deg}$ + "
        elif deg == 1:
            if abs(c) > 1e-10:
                label += f"{c:.1f}${xlabel}$ + "
        else:
            label += f"{c:.1f}"
    return label

In [ ]:
from scipy.optimize import curve_fit
from lifters.plotting_tools import savefig

plot_labels = ["variables", "redundant constraints", "moment constraints", "total constraints"]
groupby = ["d"]
x_name = "landmarks"
x_label = "K"

for name in ["stereo_study", "stereo_study_lasserre"]:
    df = pd.read_pickle(f"../_results/{name}.pkl") # added extra constraints

    df["total constraints"] = df["found"]
    df["moment constraints"] = df["substitutions"]
    df["redundant constraints"] = df["total constraints"] - df["moment constraints"]
    df.drop(["found", "substitutions", "known"], axis=1, inplace=True)

    #display(df)
    
    for group, df_group in df.groupby(groupby):
        display(df_group)
        x = df_group[x_name].unique()

        fig, ax = plt.subplots()
        for i, label in enumerate(plot_labels):
            y = np.r_[df_group[label]]
            ax.plot(x, y, label=label, color=f"C{i+1}")

            coeffs = np.polyfit(x, y, 2)
            #coeffs = np.round(coeffs, 0)
            poly = np.poly1d(coeffs)
            label= poly_label(poly, xlabel=x_label) #f"{coeffs[0]:.1f}$x^2$ + {coeffs[1]:.1f}$x$ + {coeffs[2]:.1f}"
            ax.plot(x, poly(x), label=label, ls="", marker='o', color=f"C{i+1}")
        ax.legend(loc="upper left")
        ax.set_xlabel(f"{x_name} ${x_label}$")
        ax.set_xticks(x)
        title=','.join([f"{v}{k}" for k, v in zip(groupby, [group])])
        ax.set_title(title)
        ax.grid()
        savefig(fig, f"../_plots/{name}_{title}.png")

# Next experiments